## Imports

In [1]:
import pandas as pd
import os
import numpy as np


## Functions 

### Find Each Table's primary key

In [2]:
def pk_finder(name, df):
    for i in range(len(df.columns)):
        col = df.iloc[:,i]
        #if(col.size == col.drop_duplicates().size):
        if(col.nunique() == col.size):
            pk[name] = df.columns[i] 
            break
        

### Find Each Table's foreign key


In [3]:
def fk_finder(name, df):
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        col = df.iloc[:,i]
        if(list(df.columns)[i] not in pk):
            flag = True
            for j in pk: #loops on the key j = table names
                if j != name:
                    primary_key_column = All_dfs[j][(pk[j])]
                    if primary_key_column.dtype == col.dtype:
                        status = col.isin(primary_key_column)
                        if status[status== False].size == 0:
                            fk[name][col_name] = j 
                            ref_in[j].add(name)
                            break

### Assign Tables to nodes or edges

In [4]:
def nodes_edges(fk):
    for f in fk:
        if len(fk[f]) == 2:
            inner_dict = list(fk[f])
            edges[f] = fk[f]
        elif len(fk[f]) == 1:
            properties[f] = fk[f]
        else:
            nodes[f] = All_dfs[f]
#         else:
#             if len(fk[f]) == 1:
#                 properties[f] = fk[f]
            
#             nodes[f] = All_dfs[f]
    # for f in fk:
    #     if((f not in edges) ):
    #         nodes[f] = All_dfs[f]
    #         #nodes.append(f)
        

### Create Nodes Table

In [5]:
def add_nodes(nodes, nodesTable):
    for n in range(len(nodes)):
        for index, row in nodes[list(nodes.keys())[n]].iterrows():
            column_names = list(nodes[list(nodes.keys())[n]].columns) #get column names
            att = {}
            for i in range(1, len(column_names)):
                att[column_names[i]] = nodes[list(nodes.keys())[n]].iloc[index, i]
            newRow = [{'Label': list(nodes.keys())[n] , 'ID': nodes[list(nodes.keys())[n]].iloc[index,0] , 'Attributes': att}]
            tmp = pd.DataFrame(newRow)
            nodesTable = pd.concat([nodesTable, tmp], ignore_index=True)
    return nodesTable


### Create Edges Table

In [6]:
def add_edges(edges,edgesTable,pk,fk,ref_in):
    for e in edges:
        df = All_dfs[e] #df of the cur edge
        for r in range(len(df)):
            label = list(edges[e].keys())
            from_id = nodesTable[(nodesTable["Label"] == edges[e][label[0]]) & (df.loc[r,label[0]] == nodesTable["ID"])]
            from_id = from_id.index[0]
            to_id = nodesTable[(nodesTable["Label"] == edges[e][label[1]]) & (df.loc[r,label[1]] == nodesTable["ID"])]
            to_id = to_id.index[0]
            pk_col = pk[e]
#             primary_key = df[pk_col].iloc[r]
            primary_key = df.loc[r,pk_col]
            att = convert_prop(ref_in,e,primary_key,fk)
            newRow = [{'From_Node_ID': from_id, 'To_Node_ID': to_id, 'order/service' : att}]
            tmp = pd.DataFrame(newRow)
            edgesTable = pd.concat([edgesTable, tmp], ignore_index=True)
    return edgesTable

### Properties of the Edge

In [7]:
def convert_prop(ref_in,edge_key, pk_value,fk):
    for referenced_table_name in ref_in[edge_key]:
        referenced_table = All_dfs[referenced_table_name]
        fk_of_ref_table = fk[referenced_table_name]
        for foreign_key in fk_of_ref_table:
            if fk_of_ref_table[foreign_key] == edge_key:
#                 referenced_table[referenced_table[foreign_key]== id]
                all_occurances_df = referenced_table[referenced_table[foreign_key]== pk_value].drop([foreign_key],axis=1)
                return all_occurances_df.to_dict('records')
                

## Global Variables

In [8]:
# {Table name: df}
All_dfs = {}
# {Table name: Table's Primary key}
pk = {}
# {Table name: {Column Name: Referenced Table}}
fk = {}
# {Table name: set(Tables that references the table)}
ref_in = {}
# {Table name: df}
nodes = {}
# {Table name: {Column Name: Referenced Table}}
edges = {}
# _
properties = {}
# Nodes_df
nodesTable = pd.DataFrame(columns=['Label', 'ID', 'Attributes'])
# Edges_df
edgesTable = pd.DataFrame(columns = ['From_Node_ID', 'To_Node_ID', 'order/service'])


## Reading Data Set

 creating a dictionary where
 key: first word of the table name 
 value: df 

In [9]:
import os
path_of_the_directory = './DataSet/'
ext = ('.csv')
for file in os.listdir(path_of_the_directory):
    if file.endswith(ext):
        print(file) 
        temp = (file.replace("_"," ").replace("."," ").split(" ")[0].lower())
        All_dfs[temp] = pd.read_csv(path_of_the_directory+file)
    else:
        continue

customer_data.csv
ExternalOrders_data.csv
ExternalServices_data.csv
ExternalShipments_data.csv
ExternalTransactions.csv
facilities_data.csv
InternalOrders_data.csv
InternalServices_data.csv
InternalShipments_data.csv
InternalTransactions.csv
Manufacturing_data.csv
Products_data.csv
Retailer_data.csv
Supplier_data.csv
warehouses_data.csv


### Splitting Shipments tables

In [10]:
SRIntShip = All_dfs["internalshipments"].query('from_to_where == "SR"')
SRIntShip = SRIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SRIntShip
RCExtShip = All_dfs["externalshipments"].query('from_to_where == "RC"')
RCExtShip = RCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#RCExtShip
SSIntShip = All_dfs["internalshipments"].query('from_to_where == "SS"')
SSIntShip = SSIntShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SSIntShip
SCExtShip = All_dfs["externalshipments"].query('from_to_where == "SC"')
SCExtShip = SCExtShip.drop(['from_to_where'], axis=1).reset_index(drop = True)
#SCExtShip
All_dfs["RCExtShip".lower()] = RCExtShip
All_dfs["SCExtShip".lower()] = SCExtShip
All_dfs["SRIntShip".lower()] = SRIntShip
All_dfs["SSIntShip".lower()] = SSIntShip

### Splitting Orders Tables

In [11]:
filter_list = All_dfs["rcextship"]["ExtShip_id"]
All_dfs["rcextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["scextship"]["ExtShip_id"]
All_dfs["scextorders"] = All_dfs["externalorders"].query("ExtShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["srintship"]["IntShip_id"]
All_dfs["srintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)
filter_list = All_dfs["ssintship"]["IntShip_id"]
All_dfs["ssintorders"] = All_dfs["internalorders"].query("IntShip_id.isin(@filter_list)").reset_index(drop=True)

### Removing Redundant Tables

In [12]:
All_dfs.pop("internalshipments")
All_dfs.pop("externalshipments")
All_dfs.pop("externalorders")
All_dfs.pop("internalorders")
All_dfs.keys()

dict_keys(['customer', 'externalservices', 'externaltransactions', 'facilities', 'internalservices', 'internaltransactions', 'manufacturing', 'products', 'retailer', 'supplier', 'warehouses', 'rcextship', 'scextship', 'srintship', 'ssintship', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Initialize fk and ref_in

In [13]:
table_name = list(All_dfs.keys())
for table_name in All_dfs:
    fk[table_name] = {}
    ref_in[table_name] = set()

### Get Primary key for each table

In [14]:
for t in All_dfs:
    pk_finder(t,All_dfs[t])
pk

{'customer': 'cust_id',
 'externalservices': 'ExtServ_id',
 'externaltransactions': 'ExtTran_id',
 'facilities': 'fac_id',
 'internalservices': 'IntServ_id',
 'internaltransactions': 'IntTran_id',
 'manufacturing': 'Different_suppliers',
 'retailer': 'retailer_id',
 'supplier': 'supp_id',
 'warehouses': 'warehouse_id',
 'rcextship': 'ExtShip_id',
 'scextship': 'ExtShip_id',
 'srintship': 'IntShip_id',
 'ssintship': 'IntShip_id',
 'rcextorders': 'ExtOrders_id',
 'scextorders': 'ExtOrders_id',
 'srintorders': 'IntOrders_id',
 'ssintorders': 'IntOrders_id'}

### Get Foreing key for each table

In [15]:
for t in All_dfs:
    fk_finder(t,All_dfs[t])
fk

{'customer': {},
 'externalservices': {'ExtTrans_id': 'externaltransactions'},
 'externaltransactions': {'CompFrom': 'supplier', 'Custto': 'customer'},
 'facilities': {'supplier_id': 'supplier'},
 'internalservices': {'IntTrans_id': 'internaltransactions'},
 'internaltransactions': {'CompFrom': 'supplier', 'Compto': 'supplier'},
 'manufacturing': {'Factory_id': 'supplier'},
 'products': {},
 'retailer': {},
 'supplier': {},
 'warehouses': {},
 'rcextship': {'factoryIds/retailerIds': 'retailer', 'idsTo': 'customer'},
 'scextship': {'factoryIds/retailerIds': 'supplier', 'idsTo': 'customer'},
 'srintship': {'listSuppIds': 'supplier', 'factoryIds': 'retailer'},
 'ssintship': {'listSuppIds': 'supplier', 'factoryIds': 'supplier'},
 'rcextorders': {'ExtShip_id': 'rcextship'},
 'scextorders': {'ExtShip_id': 'scextship'},
 'srintorders': {'IntShip_id': 'srintship'},
 'ssintorders': {'IntShip_id': 'ssintship'}}

In [16]:
ref_in

{'customer': {'externaltransactions', 'rcextship', 'scextship'},
 'externalservices': set(),
 'externaltransactions': {'externalservices'},
 'facilities': set(),
 'internalservices': set(),
 'internaltransactions': {'internalservices'},
 'manufacturing': set(),
 'products': set(),
 'retailer': {'rcextship', 'srintship'},
 'supplier': {'externaltransactions',
  'facilities',
  'internaltransactions',
  'manufacturing',
  'scextship',
  'srintship',
  'ssintship'},
 'warehouses': set(),
 'rcextship': {'rcextorders'},
 'scextship': {'scextorders'},
 'srintship': {'srintorders'},
 'ssintship': {'ssintorders'},
 'rcextorders': set(),
 'scextorders': set(),
 'srintorders': set(),
 'ssintorders': set()}

### Determine nodes and edges

In [17]:
nodes_edges(fk)

In [18]:
nodes.keys()

dict_keys(['customer', 'products', 'retailer', 'supplier', 'warehouses'])

In [19]:
edges.keys()

dict_keys(['externaltransactions', 'internaltransactions', 'rcextship', 'scextship', 'srintship', 'ssintship'])

In [20]:
properties.keys()

dict_keys(['externalservices', 'facilities', 'internalservices', 'manufacturing', 'rcextorders', 'scextorders', 'srintorders', 'ssintorders'])

### Create NodesTable

In [21]:
nodesTable = add_nodes(nodes,nodesTable)
nodesTable

,Label,ID,Attributes
0,customer,33736,"{'gender': 'F', 'first_name': 'Nadia', 'last_n..."
1,customer,42959,"{'gender': 'M', 'first_name': 'Eric', 'last_na..."
2,customer,50653,"{'gender': 'F', 'first_name': 'Alessia', 'last..."
3,customer,82485,"{'gender': 'M', 'first_name': 'Alexei', 'last_..."
4,customer,56609,"{'gender': 'F', 'first_name': 'Brenda', 'last_..."
...,...,...,...
1195,warehouses,4254,"{'country': 'Malaysia', 'city_name': 'Wilayah ..."
1196,warehouses,6678,"{'country': 'Morocco', 'city_name': 'Tata', 'c..."
1197,warehouses,1231,"{'country': 'Greece', 'city_name': 'Ágion Óros..."
1198,warehouses,4712,"{'country': 'Cuba', 'city_name': 'Matanzas', '..."


### Create Edges Table

In [22]:
edgesTable = add_edges(edges, edgesTable,pk,fk,ref_in)
edgesTable

,From_Node_ID,To_Node_ID,order/service
0,984,140,[]
1,926,72,"[{'ExtServ_id': 8658, 'placed_when': '1998-06-..."
2,965,2,[]
3,874,137,[]
4,975,92,"[{'ExtServ_id': 7764, 'placed_when': '1973-08-..."
...,...,...,...
395,959,848,"[{'IntOrders_id': 2751, 'quantity': 734, 'plac..."
396,821,971,"[{'IntOrders_id': 1084, 'quantity': 784, 'plac..."
397,843,851,"[{'IntOrders_id': 4939, 'quantity': 819, 'plac..."
398,821,894,"[{'IntOrders_id': 6448, 'quantity': 712, 'plac..."


In [23]:
All_dfs["facilities"].fac_id.value_counts()

694420    1
894911    1
990810    1
629057    1
698102    1
         ..
643667    1
434354    1
419150    1
542930    1
186890    1
Name: fac_id, Length: 2000, dtype: int64

In [25]:
facilities = properties['facilities']
facilities

{'supplier_id': 'supplier'}